In [ ]:
import os
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Activation, Input, Add, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, ZeroPadding2D

In [50]:
def set_seed():
  InitSeed = 767
  tf.random.set_seed(InitSeed)
  np.random.seed( InitSeed)


def one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y


tf.config.run_functions_eagerly(True)
(x_train_full, y_train_full), (x_test, y_test) = cifar10.load_data()
x_train, x_valid = x_train_full[:-5000]/255, x_train_full[-5000:]/255
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]
x_train = x_train[..., np.newaxis]
x_valid = x_valid[..., np.newaxis]
x_test = x_test[..., np.newaxis]

y_train = one_hot(y_train, 10).T
y_valid = one_hot(y_valid, 10).T
y_test = one_hot(y_test, 10).T
print ("X_train shape: " + str(x_train.shape)  + 
    "\nY_train shape: " + str(y_train.shape) + 
    "\nX_test shape: " + str(x_test.shape)  + 
    "\nY_test shape: " + str(y_test.shape)  +
    "\nX_valid shape: " + str(x_valid.shape) + 
    "\nY_valid shape: " + str(y_valid.shape))

X_train shape: (45000, 32, 32, 3, 1)
Y_train shape: (45000, 10)
X_test shape: (10000, 32, 32, 3, 1)
Y_test shape: (10000, 10)
X_valid shape: (5000, 32, 32, 3, 1)
Y_valid shape: (5000, 10)


In [ ]:
#
def block(X, filters, first_block=False):
  """
  X: input
  filters: List, dim of filters in [f1, f2, f3]
  first_block: Bool, if this block is the first block of this big "layer"

  return: the output X went through all layers in this block
  """
  # Receive filter dimension information
  f1, f2, f3 = filters
  output_cut = X

  # By using functional API
  # First layer
  if first_block: 
    output_ = Conv2D(filters=f1, kernel_size=(1,1), strides=(2,2), padding='valid')(X)
  else:
    output_ = Conv2D(filters=f1, kernel_size=(1,1), strides=(1,1), padding='valid')(X)
  output_ = BatchNormalization(axis=3)(output_)
  output_ = Activation('relu')(output_)
  
  # Second layer
  output_ = Conv2D(filters=f2, kernel_size=(3,3), strides=(1,1), padding='same')(output_)
  output_ = BatchNormalization(axis=3)(output_)
  output_ = Activation('relu')(output_)

  # Third layer
  output_ = Conv2D(filters=f3, kernel_size=(1,1), strides=(1,1), padding='valid')(output_)
  output_ = BatchNormalization(axis=3)(output_)

  if first_block:
    output_cut = Conv2D(filters=f3, kernel_size=(1,1), strides=(2,2), padding='valid')(output_cut)
    output_cut = BatchNormalization(axis=3)(output_cut)

  # "Addition step" - skip-connection value merges with main path
  # NOTE: both values have same dimensions at this point, so no operation is required to match dimensions
  output_ = Add()([output_, output_cut])
  output_ = Activation('relu')(output_)

  return output_

In [45]:
# Start creating the network
# Since cifar10 is 32x32x3, 10 classes output,
def resNet50(dim=(32,32,3), label=10):
  # Receive input
  X = Input(dim)
  output_ = ZeroPadding2D((3, 3))(X)
  
  # ================ 7x7, 64, stride 2 ================ 
  output_ = Conv2D(64, (7, 7), strides = (2, 2))(output_)
  output_ = BatchNormalization(axis = 3)(output_)
  output_ = Activation('relu')(output_)

  # ============= 3x3 max pool, stride 2 =============
  output_ = MaxPooling2D((3, 3), strides=(2, 2))(output_)

  # ======================= Blocks =======================
  """
  |1 x 1, 64.|
  |3 x 3, 64.| X 3
  |1 x 1, 256|"""
  output_ = block(output_, filters=[64, 64, 256], first_block=True)
  output_ = block(output_, filters=[64, 64, 256], first_block=False)
  output_ = block(output_, filters=[64, 64, 256], first_block=False)

  """
  |1 x 1, 128|
  |3 x 3, 128| X 4
  |1 x 1, 512|
  """
  output_ = block(output_, filters=[128, 128, 512], first_block=True)
  for i in range(3):
    output_ = block(output_, filters=[128, 128, 512], first_block=False)

  """
  |1 x 1, 256.|
  |3 x 3, 256.| X 6
  |1 x 1, 1024|
  """
  output_ = block(output_, filters=[256, 256, 1024], first_block=True)
  for i in range(5):
    output_ = block(output_, filters=[256, 256, 1024], first_block=False)

  """
  |1 x 1, 512.|
  |3 x 3, 512.| X 3
  |1 x 1, 2048|
  """
  output_ = block(output_, filters=[512, 512, 2048], first_block=True)
  output_ = block(output_, filters=[512, 512, 2048], first_block=False)
  output_ = block(output_, filters=[512, 512, 2048], first_block=False)

  # average pooling, softmax
  output_ = AveragePooling2D((2, 2), padding='same')(output_)
  output_ = Flatten()(output_)
  output_ = Dense(label, activation='softmax', )(output_)

  # Aggregate as a model
  model = Model(inputs=X, outputs=output_)
  return model

In [54]:
set_seed()
model = resNet50(dim=(32, 32, 3), label=10)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [55]:
model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=5)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


1407/1407 [==============================] - 323s 229ms/step - loss: 2.3738 - accuracy: 0.2708 - val_loss: 1.8962 - val_accuracy: 0.3276
Epoch 2/5
1407/1407 [==============================] - 322s 229ms/step - loss: 2.2576 - accuracy: 0.2846 - val_loss: 2.6578 - val_accuracy: 0.2084
Epoch 3/5
1407/1407 [==============================] - 321s 228ms/step - loss: 2.3133 - accuracy: 0.2504 - val_loss: 12.5751 - val_accuracy: 0.2676
Epoch 4/5
1407/1407 [==============================] - 320s 228ms/step - loss: 1.8977 - accuracy: 0.3641 - val_loss: 3.8347 - val_accuracy: 0.1540
Epoch 5/5
1407/1407 [==============================] - 319s 227ms/step - loss: 1.9654 - accuracy: 0.3689 - val_loss: 3.3166 - val_accuracy: 0.3242
